# SentenceTransformers SBERT

https://www.sbert.net/index.html


## getting started

In [25]:
import numpy as np
import pandas as pd
import pickle

from sentence_transformers import SentenceTransformer, util

In [2]:
## Mult-lingual model that supports Dutch
modelML = SentenceTransformer('xlm-r-100langs-bert-base-nli-mean-tokens')

## import kamer speeches

In [ ]:
## put data chuncks in one zip file
!cat data/CorpusTweedeKamer* > CorpusTweedeKamer.zip

In [5]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 12.9 s, sys: 1 s, total: 13.9 s
Wall time: 14.7 s


In [6]:
recente_speeches = (
    tweede_kamer
    .dropna(subset = ["text"])
    .query('terms > 75')
    .query('terms < 800')
    .query('datum > "2016-01-01"')
)

In [18]:
recente_speeches.shape

(95314, 12)

In [19]:
speeches = recente_speeches.text.str.lower().values.tolist()

In [77]:
%%time 
sentence_embeddings = modelML.encode(speeches)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [62]:
sentence_embeddings.shape

(95314, 768)

In [26]:
#Store sentences & embeddings on disc
with open('kamer_BERT_embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': speeches[0:5000], 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)



In [61]:
#Load sentences & embeddings from disc
with open('kamer_BERT_embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    sentences = stored_data['sentences']
    sentence_embeddings = stored_data['embeddings']

### Voer een semantic search uit

In [72]:
tt = "doe eens normaal "

In [73]:
query_embedding = modelML.encode(tt, convert_to_tensor=True)
topmatches = util.semantic_search(query_embedding, sentence_embeddings)

In [74]:
topmatches

[[{'corpus_id': 68748, 'score': 0.7371483},
  {'corpus_id': 36305, 'score': 0.717166},
  {'corpus_id': 77454, 'score': 0.66476923},
  {'corpus_id': 64533, 'score': 0.6506115},
  {'corpus_id': 34866, 'score': 0.6485617},
  {'corpus_id': 69774, 'score': 0.64739126},
  {'corpus_id': 22877, 'score': 0.63766456},
  {'corpus_id': 46543, 'score': 0.635753},
  {'corpus_id': 82126, 'score': 0.62685025},
  {'corpus_id': 65001, 'score': 0.62231666}]]

In [75]:
top_textst = [ idx['corpus_id']  for idx in topmatches[0]]
top_textst

[68748, 36305, 77454, 64533, 34866, 69774, 22877, 46543, 82126, 65001]

In [76]:
[speeches[i] for i in top_textst]

['ik weet niet of dat in een demografisch onderzoek past, maar dat is wel het beleid van deze regering. het kabinet doet dat door volstrekt helder te zijn over wat onze normen en waarden zijn. wij realiseren ons ook het volgende. als een economische migrant uit afrika hiernaartoe komt, dan heeft die hier in principe helemaal niets te zoeken. wegwezen. als een vluchteling uit afrika hiernaartoe komt die hier ook als vluchteling mag zijn, dan gaan we ons best doen voor de integratie. we weten allemaal dat dit bij een vluchteling uit een meer westers georiënteerd land — noem eens een land — die wellicht ook al engels spreekt veel sneller zal gaan dan bij een vluchteling uit syrië met een lage opleiding. ik wil alleen voorkomen dat we daarbij één aspect van de identiteit van die persoon centraal stellen, namelijk het geloof. ik denk dat er zoveel meer aspecten zijn die bijdragen aan de complexiteit van de integratievraag bij het ene en het andere geval.',
 'ik dacht dat er één meetpunt was